In [2]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
#from qiskit_aer.noise import NoiseModel
#from qiskit_ibm_runtime import QiskitRuntimeService
#from qiskit.primitives import BackendSamplerV2
import numpy as np

In [3]:
backend= AerSimulator()

In [4]:
def function_tel_tomography(n=0, shots=1000):
    # Registradores quânticos
    Alice = QuantumRegister(1, 'Alice')
    Bob = QuantumRegister(1, 'Bob')
    Secret = QuantumRegister(1, 'Secret')
    c = ClassicalRegister(3, 'c')  # c[0] = Secret, c[1] = Alice, c[2] = tomografia

    if n > 0:
        ext = QuantumRegister(n, 'ext')
        circuit = QuantumCircuit(Secret, Alice, Bob, ext, c)
    else:
        circuit = QuantumCircuit(Secret, Alice, Bob, c)

    # EPR
    circuit.h(Alice)
    circuit.cx(Alice, Bob)
    circuit.barrier()

    # Swaps
    if n > 0:
        circuit.swap(Bob[0], ext[0])
        for i in range(1, n):
            circuit.swap(ext[i - 1], ext[i])
        final_target = ext[n - 1]
    else:
        final_target = Bob[0]

    # Estado secreto: H|0> = |+>
    circuit.barrier()
    circuit.h(Secret)
    circuit.barrier()

    # Entrelançamento e medições
    circuit.cx(Secret, Alice)
    circuit.h(Secret)
    circuit.barrier()

    circuit.measure(Alice, c[0])
    circuit.measure(Secret, c[1])
    circuit.barrier()

    # Correções condicionais baseadas nas medições
    with circuit.if_test((c[1], 1)):  # Se Alice medir 1, aplicar Z
        circuit.z(final_target)

    with circuit.if_test((c[0], 1)):  # Se Secret medir 1, aplicar X
        circuit.x(final_target)

    # =============== TOMOGRAFIA DIRETA ===============
    pauli_list = [
        np.eye(2),
        np.array([[0.0, 1.0], [1.0, 0.0]]),       # X
        np.array([[0, -1.0j], [1.0j, 0.0]]),      # Y
        np.array([[1.0, 0.0], [0.0, -1.0]])       # Z
    ]
    expectation_values = {}
    all_counts = {}

    for base in ['X', 'Y', 'Z']:
        circ = circuit.copy()

        # Aplica a rotação de base antes da medição
        if base == 'X':
            circ.h(final_target)
        elif base == 'Y':
            circ.sdg(final_target)
            circ.h(final_target)

        # Mede o qubit final no registrador c[2]
        circ.measure(final_target, c[2])

        transpiled = transpile(circ, backend)
        result = backend.run(transpiled, shots=shots).result()
        counts = result.get_counts()

        target_classical_index = 2  # ou outro índice que você usou no `measure(final_target, c[2])`
        p0 = sum(cnt for bits, cnt in counts.items() if bits[::-1][target_classical_index] == '0')
        p1 = sum(cnt for bits, cnt in counts.items() if bits[::-1][target_classical_index] == '1')


        total = p0 + p1 or 1
        expectation_values[base] = (p0 - p1) / total
        all_counts[base] = counts

    # Reconstrução da matriz densidade
    rho = 0.5 * (
        pauli_list[0]
        + expectation_values['X'] * pauli_list[1]
        + expectation_values['Y'] * pauli_list[2]
        + expectation_values['Z'] * pauli_list[3]
    )

    return rho, expectation_values, circuit,all_counts,expectation_values


In [5]:
rho, expectation_values, circuit,all_counts,expectation_values = function_tel_tomography(n=20,shots=1024)
print("Matriz densidade reconstruída:")
print(rho)

print("Valores esperados:")


Matriz densidade reconstruída:
[[0.5078125+0.j         0.5      -0.00585938j]
 [0.5      +0.00585938j 0.4921875+0.j        ]]
Valores esperados:


In [6]:
print(expectation_values)

{'X': 1.0, 'Y': 0.01171875, 'Z': 0.015625}


In [7]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator
import numpy as np

backend = AerSimulator()

def measure_in_basis(circuit, final_target, base, shots=1024):
    """
    Mede o qubit final na base especificada (X, Y ou Z) e retorna o valor esperado
    e o dicionário de contagens. Extraímos o token do registrador de tomografia
    a partir das chaves de counts (como '0 10' etc).
    """
    circ = circuit.copy()
    
    # Registrador clássico apenas para tomografia (1 bit)
    c_temp = ClassicalRegister(1, 'c_temp')
    circ.add_register(c_temp)

    # Rotação de base
    if base == 'X':
        circ.h(final_target)
    elif base == 'Y':
        circ.sdg(final_target)
        circ.h(final_target)
    # se base == 'Z' não faz nada

    # Mede o qubit final no registrador temporário
    circ.measure(final_target, c_temp[0])

    transpiled = transpile(circ, backend)
    result = backend.run(transpiled, shots=shots).result()
    counts = result.get_counts()

    # Extrai o token de 1 bit dentro da chave (cada chave pode ter tokens separados por espaço)
    p0 = 0
    p1 = 0
    for key, cnt in counts.items():
        # chave ex: '0 10' ou '1 01' etc
        tokens = key.split()  # separa por espaços; cada token = um registrador clássico
        # tenta achar token com length == 1 (nosso c_temp)
        # caso haja múltiplos tokens de length 1, pegamos o primeiro (normalmente só há um)
        single_bit_token = None
        for t in tokens:
            if len(t) == 1:
                single_bit_token = t
                break
        if single_bit_token is None:
            # fallback: se não há tokens separados por espaço, pode ser que
            # a string seja algo como '010' — então pegamos o último bit (mais comum)
            bit = key[-1]
        else:
            bit = single_bit_token[-1]  # token tem tamanho 1, então bit == token
        if bit == '0':
            p0 += cnt
        elif bit == '1':
            p1 += cnt
        else:
            # caso estranho: ignore
            pass

    total = p0 + p1 or 1
    expectation = (p0 - p1) / total
    # Construímos um `counts_simplified` para facilitar debug: {'0': p0, '1': p1}
    counts_simplified = {'0': p0, '1': p1}
    return expectation, counts_simplified

def function_tel_tomography(n=0, shots=1000):
    # Registradores
    Alice = QuantumRegister(1, 'Alice')
    Bob = QuantumRegister(1, 'Bob')
    Secret = QuantumRegister(1, 'Secret')
    c = ClassicalRegister(2, 'c')  # apenas para medições de Alice e Secret

    if n > 0:
        ext = QuantumRegister(n, 'ext')
        circuit = QuantumCircuit(Secret, Alice, Bob, ext, c)
    else:
        circuit = QuantumCircuit(Secret, Alice, Bob, c)

    # EPR
    circuit.h(Alice)
    circuit.cx(Alice, Bob)
    circuit.barrier()

    # Swaps (linha)
    if n > 0:
        circuit.swap(Bob[0], ext[0])
        for i in range(1, n):
            circuit.swap(ext[i - 1], ext[i])
        final_target = ext[n - 1]
    else:
        final_target = Bob[0]

    # Estado secreto |+>
    circuit.barrier()
    circuit.h(Secret)
    circuit.barrier()

    # Medida de Bell
    circuit.cx(Secret, Alice)
    circuit.h(Secret)
    circuit.barrier()

    # Medições de Alice e Secret (registrador c tem 2 bits)
    circuit.measure(Alice, c[0])
    circuit.measure(Secret, c[1])
    circuit.barrier()

    # Correções condicionais
    with circuit.if_test((c[1], 1)):
        circuit.z(final_target)
    with circuit.if_test((c[0], 1)):
        circuit.x(final_target)

    # TOMOGRAFIA: medições separadas (cada uma chamando measure_in_basis)
    ex_X, counts_X = measure_in_basis(circuit, final_target, 'X', shots)
    ex_Y, counts_Y = measure_in_basis(circuit, final_target, 'Y', shots)
    ex_Z, counts_Z = measure_in_basis(circuit, final_target, 'Z', shots)

    expectation_values = {'X': ex_X, 'Y': ex_Y, 'Z': ex_Z}
    all_counts = {'X': counts_X, 'Y': counts_Y, 'Z': counts_Z}

    pauli_list = [
        np.eye(2),
        np.array([[0.0, 1.0], [1.0, 0.0]]),       # X
        np.array([[0, -1.0j], [1.0j, 0.0]]),      # Y
        np.array([[1.0, 0.0], [0.0, -1.0]])       # Z
    ]

    rho = 0.5 * (
        pauli_list[0]
        + ex_X * pauli_list[1]
        + ex_Y * pauli_list[2]
        + ex_Z * pauli_list[3]
    )

    return rho, expectation_values, circuit, all_counts


In [8]:
rho_20, exps_20, circ_20, counts_20 = function_tel_tomography(n=20, shots=1024)

print("Matriz densidade reconstruída (n=20):")
print(rho_20)
print("\nValores esperados:", exps_20)
print("\nContagens:", counts_20)

Matriz densidade reconstruída (n=20):
[[0.52441406+0.j         0.5       -0.00292969j]
 [0.5       +0.00292969j 0.47558594+0.j        ]]

Valores esperados: {'X': 1.0, 'Y': 0.005859375, 'Z': 0.048828125}

Contagens: {'X': {'0': 1024, '1': 0}, 'Y': {'0': 515, '1': 509}, 'Z': {'0': 537, '1': 487}}
